In [10]:
import pandas as pd
import json
from pyelasticsearch import ElasticSearch
import nltk 
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import re
from textblob import TextBlob

In [6]:
data = pd.read_json('deathfx.json')

def colNames(data):
    colNames = (list(data.columns.values))
    return colNames

colNames(data)    

data['MessageWords'] = ""
data['DescriptionWords'] = ""
data['MessageSenti'] = ""
data['DescriptionSenti'] = ""
data["url"] = ""

data["url"][data["Description"].str.startswith("https")] = data["Description"]

c:\anaconda2\envs\death\lib\site-packages\ipykernel\__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [26]:
text = "Texas Department of Justice"
TextBlob(text).sentiment

Sentiment(polarity=0.0, subjectivity=0.0)

In [5]:
def review_to_words( raw_review ):
    review_text = BeautifulSoup(raw_review).get_text() 
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    words = letters_only.lower().split()                             
    stops = set(stopwords.words("english"))                  
    meaningful_words = [w for w in words if not w in stops]   
    return( " ".join( meaningful_words ))  



In [15]:
dataLen = len(data)

for i in range (0,dataLen):
    data['MessageWords'][i] = review_to_words(data["Message"][i])
    data['DescriptionWords'][i] = review_to_words(data["Description"][i])
    data['MessageSenti'][i] = TextBlob(data["MessageWords"][i]).sentiment[0]
    data['DescriptionSenti'][i] = TextBlob(data["DescriptionWords"][i]).sentiment[0]
    

c:\anaconda2\envs\death\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\anaconda2\envs\death\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\anaconda2\envs\death\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\anaconda2\envs\death\lib\site-packages\ipykernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

In [8]:
d = {}

def mapping():
    for i in colNames(data):
        a = raw_input('Do you want to analyze ' + i + ' ?\t')
        if a == 'y':
            d[i] = {'type': raw_input('.. and data type?')}
        else:
            d[i]= {'index':'not_analyzed','type': raw_input('.. and data type?\t')}
    return d

settings = {
    'records':{
        'properties':
            mapping()
    }
}




Do you want to analyze Age ?	n
.. and data type?	integer
Do you want to analyze County ?	n
.. and data type?	string
Do you want to analyze Date ?	n
.. and data type?	date
Do you want to analyze Description ?	y
.. and data type?string
Do you want to analyze Message ?	y
.. and data type?string
Do you want to analyze Mid ?	n
.. and data type?	integer
Do you want to analyze Race ?	n
.. and data type?	string
Do you want to analyze firstName ?	n
.. and data type?	string
Do you want to analyze lastName ?	n
.. and data type?	string
Do you want to analyze FilteredMessage ?	y
.. and data type?string
Do you want to analyze url ?	n
.. and data type?	string


In [9]:
settings

{'records': {'properties': {u'Age': {'index': 'not_analyzed',
    'type': 'integer'},
   u'County': {'index': 'not_analyzed', 'type': 'string'},
   u'Date': {'index': 'not_analyzed', 'type': 'date'},
   u'Description': {'type': 'string'},
   'FilteredMessage': {'type': 'string'},
   u'Message': {'type': 'string'},
   u'Mid': {'index': 'not_analyzed', 'type': 'integer'},
   u'Race': {'index': 'not_analyzed', 'type': 'string'},
   u'firstName': {'index': 'not_analyzed', 'type': 'string'},
   u'lastName': {'index': 'not_analyzed', 'type': 'string'},
   'url': {'index': 'not_analyzed', 'type': 'string'}}}}

In [10]:
es = ElasticSearch('localhost:9200/')

index = "death"
doc = "death.a"

In [11]:
try :
    es.delete_index(index)
except :
    pass

es.create_index(index, settings={'mappings': settings})

{u'acknowledged': True}

In [12]:
records = data.where(pd.notnull(data), None).T.to_dict()

In [13]:
list_records = [records[it] for it in records]

In [14]:
es.bulk_index(index,doc,list_records)

{u'errors': False,
 u'items': [{u'create': {u'_id': u'AVTjz92WI1uIEsvZIOr7',
    u'_index': u'death',
    u'_shards': {u'failed': 0, u'successful': 1, u'total': 2},
    u'_type': u'death.a',
    u'_version': 1,
    u'status': 201}},
  {u'create': {u'_id': u'AVTjz92WI1uIEsvZIOr8',
    u'_index': u'death',
    u'_shards': {u'failed': 0, u'successful': 1, u'total': 2},
    u'_type': u'death.a',
    u'_version': 1,
    u'status': 201}},
  {u'create': {u'_id': u'AVTjz92WI1uIEsvZIOr9',
    u'_index': u'death',
    u'_shards': {u'failed': 0, u'successful': 1, u'total': 2},
    u'_type': u'death.a',
    u'_version': 1,
    u'status': 201}},
  {u'create': {u'_id': u'AVTjz92WI1uIEsvZIOr-',
    u'_index': u'death',
    u'_shards': {u'failed': 0, u'successful': 1, u'total': 2},
    u'_type': u'death.a',
    u'_version': 1,
    u'status': 201}},
  {u'create': {u'_id': u'AVTjz92WI1uIEsvZIOr_',
    u'_index': u'death',
    u'_shards': {u'failed': 0, u'successful': 1, u'total': 2},
    u'_type': u'dea